# Experiment with finding circles in the Rett Girls videos

July 2018

This attempts boundary detection. I'm using a line fit for detecting colors in dots.

In [ ]:
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'
# %config InlineBackend.print_figure_kwargs={'bbox_inches':None}
import cv2
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
plt.rcParams["animation.html"] = "jshtml"

mpl.rcParams['figure.dpi']= 300

In [ ]:
vid = '/home/gb/Dropbox/Karen and Gary Shared Files/Videos & Transcripts/MSB/MSB_Video 1 (09-30-17).mp4'

osp.exists(vid)

In [ ]:
vc = cv2.VideoCapture(vid)
vc.get(cv2.CAP_PROP_FRAME_COUNT), vc.get(cv2.CAP_PROP_FPS), vc.get(cv2.CAP_PROP_FRAME_WIDTH), vc.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [ ]:
def show(im, **kwargs):
    '''Show images actual size unless it is tiny
    
    I'm assuming they are in rgb float32 if the rank is 3
    
    '''
    height, width = im.shape[:2]
    if height > 50 and width > 50:
        dpi = 100
        margin= 50
        figsize=((width+2*margin)/dpi, (height+2*margin)/dpi) # inches
        left = margin/dpi/figsize[0] #axes ratio
        bottom = margin/dpi/figsize[1]

        fig = plt.figure(figsize=figsize, dpi=dpi)
        fig.subplots_adjust(left=left, bottom=bottom, right=1.-left, top=1.-bottom)
    else:
        plt.figure()
    
    args = dict(kwargs)
    if 'title' in args:
        del args['title']
    
    if len(im.shape) == 2:
        args['cmap'] = 'gray'                  

    plt.imshow(im, **args)
    if 'title' in kwargs:
        plt.title(kwargs['title'])

In [ ]:
def grabFrame(fn):
    vc.set(cv2.CAP_PROP_POS_FRAMES, fn)
    rval, im = vc.read()
    im = cv2.cvtColor(im.astype(np.float32)/255.0, cv2.COLOR_BGR2RGB)
    return im
show(grabFrame(100))

In [ ]:
def isBlue(im):
    # these are from svd fit
    mblue = np.array([ 0.28746566,  0.70905238,  0.87152636], dtype=np.float32)
    dblue = np.array([ 0.86527359,  0.46497536,  0.18734868], dtype=np.float32)
    dim = im - mblue
    u = dim.dot(dblue)
    pim = u[:,:,None] * dblue[None,:] + mblue
    d2 = np.sum((im - pim) ** 2, axis=2)
    return np.exp(-d2 / 0.2)

im = grabFrame(800)
show(im)
pblue = isBlue(im)
print(np.max(pblue), np.min(pblue))
show(pblue)
_, tblue = cv2.threshold(pblue, 0.99, 255, cv2.THRESH_BINARY)
show(tblue)

In [ ]:
vc.set(cv2.CAP_PROP_POS_FRAMES, 100)
rval, im = vc.read()
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
# im = cv2.GaussianBlur(im, (3, 3), 0)
show(im, interpolation=None)

In [ ]:
lim = cv2.cvtColor(im, cv2.COLOR_RGB2LAB).astype(np.float32)
spotBlue = np.array([154, 117,  87], dtype=np.float32) # special blue
plt.imshow(im, interpolation=None)
print(lim[420,62])

In [ ]:
inside = np.sqrt(np.sum((lim - spotBlue)**2, axis=2)) < 25
inside = inside.astype(np.uint8)
inside = cv2.dilate(inside, np.ones((3,3), dtype=np.uint8))
show(inside, cmap='gray')
plt.grid('on')

In [ ]:
im2, contours, hierarchy = cv2.findContours(inside, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


contours_circles = []

contours_area = []
# calculate area and filter into new array
for con in contours:
    area = cv2.contourArea(con)
    # print(area)
    if 50 < area < 500:
        contours_area.append(con)
        
#plt.figure()
#aim = im.copy()
#cv2.drawContours(aim, contours_area, -1, (0,0,0), 3)
#plt.imshow(aim[300:,:200])
#show(aim)

# check if contour is of circular shape
bad_contours = []
for con in contours_area:
    perimeter = cv2.arcLength(con, True)
    area = cv2.contourArea(con)
    if perimeter == 0:
        print('zerop', con)
        break
    circularity = 4*np.pi*(area/(perimeter*perimeter))
    M = cv2.moments(con)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    print(circularity, cX, cY, area, perimeter)

    if 0.7 < circularity < 1.2:
        contours_circles.append(con)
    else:
        bad_contours.append(con)

oim = im.copy()
cv2.drawContours(oim, contours_circles, -1, (0,0,0), 1)
cv2.drawContours(oim, bad_contours, -1, (255,0,0), 1)
plt.figure()
#plt.imshow(oim[300:,:200])
show(oim)
len(contours_circles)

## get the circles

In [ ]:
oim = im.copy()
for con in contours_circles:
    (x, y), radius = cv2.minEnclosingCircle(con)
    center = (int(x+0.5), int(y+0.5))
    radius = int(radius+0.5)
    cv2.circle(oim,center,radius,(0,0,0),1)
show(oim)


## optimize the blue color

In [ ]:
rows,cols = np.where(inside)
colors = lim[rows, cols]
distances = np.sqrt(np.sum((colors - spotBlue)**2, axis=1))
plt.plot(sorted(distances))

## The error seems to take off after about 600 of them

Get the mean and std of those.

In [ ]:
ndx = np.argsort(distances)
goodcolors = colors[ndx[:600]]
mblue = np.mean(goodcolors, axis=0)
sblue = np.std(goodcolors, axis=0)

print(spotBlue, mblue, sblue)

## optimize the text color

In [ ]:
text = np.array([217, 116, 108], dtype=np.float32)
rows,cols = np.where(inside)
colors = lim[rows, cols]
distances = np.sqrt(np.sum((colors - text)**2, axis=1))
plt.plot(sorted(distances))

## The error seems to take off after about 200 of them

Get the mean and std of those.

In [ ]:
ndx = np.argsort(distances)
goodcolors = colors[ndx[:200]]
mtext = np.mean(goodcolors, axis=0)
stext = np.std(goodcolors, axis=0)

print(text, mtext, stext)

In [ ]:
def isBlue(lim):
    mblue = np.array([154, 116, 92], dtype=np.float32)
    sblue = np.array([10.4, 3.2, 8.5], dtype=np.float32)
    blue = np.ones(lim.shape[:2], dtype=np.uint8)

    for i in range(3):
        blue &= np.abs(lim[:,:,i] - mblue[i]) < 2*sblue[i]

    return blue

def isText(lim):
    mtext = np.array([217, 116, 108], dtype=np.float32)
    stest = np.array([13.9, 6.4, 8.1], dtype=np.float32)
    text = np.ones(lim.shape[:2], dtype=np.uint8)

    for i in range(3):
        text &= np.abs(lim[:,:,i] - mtext[i]) < stext[i]

    return text

In [ ]:
show(isBlue(lim))

In [ ]:
def overlap(c1, others):
    for c2 in others:
        for p in c2:
            if cv2.pointPolygonTest(c1, tuple(p[0]), False) > 0:
                return True
    return False

In [ ]:
def isCircular(contour):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    area = cv2.contourArea(contour)
    circularity = 4 * np.pi * (area / perimeter ** 2)
    return 0.7 <= circularity <= 1.2
    #return 0.5 <= circularity <= 1.6

def findFixations(image, changed, verbose=False):
    # convert to lab space
    lim = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    # find the special blue
    blue = isBlue(lim)
    # dilate a bit to fill in noise
    blue = cv2.dilate(blue, np.ones((3,3), dtype=np.uint8))
    blue = cv2.erode(blue, np.ones((3,3), dtype=np.uint8))
    blue &= changed
    
    if verbose:
        show(blue)
    # get the contours of the blue regions
    im2, contours, hierarchy = cv2.findContours(blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if verbose:
        oim = blue.copy()
        cv2.drawContours(oim, contours, -1, (255,0,0), 1)
        show(oim)
    # filter by area
    if True:
        minArea = np.pi * 4**2
        maxArea = np.pi * 16**2
        contours = [contour for contour in contours if minArea < cv2.contourArea(contour) < maxArea]
        if verbose:
            oim = blue.copy()
            cv2.drawContours(oim, contours, -1, (255,0,0), 1)
            show(oim)
    if True:
        # filter by circularity
        contours = [contour for contour in contours if isCircular(contour)]
        if verbose:
            oim = blue.copy()
            cv2.drawContours(oim, contours, -1, (255,0,0), 1)
            show(oim)
    # return the contours
    #circles = [cv2.minEnclosingCircle(contour) for contour in contours]
    return contours

# page 4 starts at 679, ends at 844
vc.set(cv2.CAP_PROP_POS_FRAMES, 780)
rval, im = vc.read()
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
fixations = findFixations(im, np.ones(im.shape[:2], dtype=np.uint8) )
print(len(fixations), 'fixations')

cv2.drawContours(im, fixations, -1, (0, 0, 0), 1)
show(im)


In [ ]:
startFrame = 780
endFrame = 790
frameFixations = []
changed = np.ones(im.shape[:2], dtype=np.uint8)
previous = None
for frame in range(startFrame, endFrame):
    if frame % 10 == 0:
        print(frame)
    vc.set(cv2.CAP_PROP_POS_FRAMES, frame)
    rval, im = vc.read()
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im32 = im.astype(np.int32)

    if previous is not None:
        delta = np.max(np.abs(im32 - previous), axis=2)
        changed = delta == 0
    previous = im32
    frameFixations.append(findFixations(im, changed, True))

In [ ]:
def center(c):
    M1 = cv2.moments(c)
    return np.array([M["m10"] / M["m00"], M["m01"] / M["m00"]])

def dist(p1, p2):
    return np.sqrt(np.sum((p1-p2)**2))

def similar(c1, c2):
    '''True if two contours are similar.'''
    if c1.size == c2.size and np.all(c1 == c2):
        return True

    if dist(center(c1), center(c2)) > 1:
        return False
    
    if abs(cv2.contourArea(c1) - cv2.contourArea(c2)) > 5:
        return False
    
    return True

newFixations = []
seen = frameFixations[0].copy() # initial set of fixations on first page
for frame in frameFixations[1:]:
    for fixation in frame:
        for f in seen:
            if similar(fixation, f):
                break
        else:
            newFixations.append(fixation)
            seen.append(fixation)
print(len(newFixations))

oim = im.copy()
cv2.drawContours(oim, newFixations, -1, (255,0,0), 2)
show(oim)

In [ ]:
frameFixations

In [ ]:
vc.set(cv2.CAP_PROP_POS_FRAMES, 800)
rval, im = vc.read()
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
#box = im[336:353,374:390]
box = im[326:363,364:400]
plt.figure()
plt.imshow(box)
lim = cv2.cvtColor(box, cv2.COLOR_RGB2LAB)
dot = lim
bdot = isBlue(dot)
plt.figure()
plt.imshow(dot)
plt.figure()
plt.imshow(bdot)

In [ ]:
dot[8,5]

In [ ]:
vc.set(cv2.CAP_PROP_POS_FRAMES, 600)
rval, im = vc.read()
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
b = findFixations(im, 8)

image = im[180:225,280:325]
show(im)

In [ ]:
radius = 8
slope = 0.1

template = makeTemplate(radius, slope, width=24)
#template = template - np.mean(template)
plt.plot(template[radius,:])
np.sum(template - np.mean(template))

In [ ]:
tscale = 0.5 
target = np.array([154, 117,  87], dtype=np.float32) # special blue
lim = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

error = np.sqrt(np.sum((lim.astype(np.float32) - target)**2, axis=2))
error = np.where(error < 10, 1, -1)
error = error.astype(np.float32)
res = cv2.matchTemplate(error, template, cv2.TM_CCORR_NORMED)
threshold = np.max(res) * tscale
print(threshold)

#show(res, cmap='gray')
#plt.grid('on')
best = getTheBest(res, threshold, radius, template.shape[0]/2)
print(best)
orig = image.copy()
for pt in best:
    rc = tuple(pt[::-1].astype(int))
    cv2.circle(orig, rc, radius, (0,0,0), 1)

show(orig)
plt.grid('on')
show(error)
plt.grid('on')
show(res, cmap='gray')
plt.grid('on')
 

In [ ]:
cv2.minMaxLoc(res)

In [ ]:
res[14,11]

In [ ]:
show(error[14:14+22,11:11+22])

In [ ]:
np.sum(error[14:14+22,11:11+22]*template)

In [ ]:
plt.imshow(error[14:14+22,11:11+22]*template)

In [ ]:
frames = []
for f in range(0, int(vc.get(cv2.CAP_PROP_FRAME_COUNT)), 1):
    vc.set(cv2.CAP_PROP_POS_FRAMES, f)
    rval, im = vc.read()
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    frames.append(im[40:60,-50:-20])
frames = np.array(frames)

In [ ]:
fig = plt.figure()
plt.imshow(frames.std(axis=0))

In [ ]:
plt.figure()
plt.figimage(im)

In [ ]:
# DPI, here, has _nothing_ to do with your screen's DPI.
dpi = 100.0
xpixels, ypixels = 854, 504

fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
ax.imshow(im)

In [ ]:
frames.shape